## 前処理用のコード 第7版

V6までは、平米あたりの単価を目的変数としていたが、土地の価格を直接目的変数にする。

In [1]:
import numpy as np
import pandas as pd
import math

In [2]:
def create_list( df,cols ):
    names = set()
    for col in cols:
        for i in range(len(df)):
            name = df.at[i,col]
            if ( type(name) != float ):
                names.add(name)
    return sorted(list(names))

In [3]:
def preprocess_genba(genba, hokakisei_cols):
    df = genba.copy()
    
    # jukyoは使用しないのでを削除
    df = df.drop('jukyo',axis=1)
    
    # 地積公募値には欠損あり(chiseki_kb_hb)
    # 実測値で補完する
    df['chiseki_kb_hb'] = df['chiseki_kb_hb'].fillna(-99)
    for i in range(len(df)):
        if ( df.at[i,'chiseki_kb_hb'] == -99 ):      
            df.at[i,'chiseki_kb_hb'] = df.at[i,'chiseki_js_hb']
    
    # 地積の公募値と実測値の差を特徴量に追加
    df['chiseki_diff'] = df['chiseki_kb_hb']-df['chiseki_js_hb']
    
    # 二つの用途から、一つの用途、建ぺい率、容積率を返却する
    # yotoはダミー変数化する
    df[['yoto','kempei','yoseki']] = df.apply(select_yoto, axis=1)
    df = pd.merge(df, pd.get_dummies(df[['yoto']],drop_first = False), left_index = True, right_index = True)
    
    # 上水種別を数値化
    df['josui'] = df.apply(lambda x: 1 if x['josui']=='公営' else 0, axis=1)
    
    # 下水種別を数値化
    df['gesui'] = df.apply(lambda x: 1 if x['gesui']=='公共下水' else 0, axis=1)
    
    # ガス種別を数値化
    # df['gas'] = df.apply(lambda x: 2 if x['gas'] =='都市ガス' else 1 if x['gas'] == '集中プロパン' else 0, axis=1)
    
    # ガスをダミー変数化
    df = pd.merge(df, pd.get_dummies(df[['gas']],drop_first = False), left_index = True, right_index = True)
    
    # 雨水（usui）をダミー変数化
    df = pd.merge(df, pd.get_dummies(df[['usui']],drop_first = False), left_index = True, right_index = True)
    
    # 欠損値処理
    # 建て売り数 0で補完（trainデータのID:726は、一件だけの販売なので、他の建て売り数は０で正しい
    df = df.fillna({'tateuri_su': 0, 'tochiuri_su':0 , 'joken_su':0 })
    
    # 同現場での階数ごとの販売数は、gotoデータとつきあわせると正確に出そうだが、とりあえず０で補完する。
    df = df.fillna({'hy1f_date_su':0, 'hy2f_date_su':0, 'hy3f_date_su':0})

    # 家屋面積(平米) (kaoku_hb)
    # 欠損しているところは、家屋有無が(無)なので、0で補完する出正しい。
    # 道路面積(mseki_rd_hb), ゴミ、公園等面積(mseki_dp_hb)は大半がゼロなのでゼロで補完しておく
    df['kaoku_hb'] = df['kaoku_hb'].fillna(0)
    df = df.fillna({'kaoku_hb':0, 'mseki_rd_hb':0, 'mseki_dp_hb':0})
    
    # 道路
    # 扱いが難しい。road1〜road4まであるが、road1だけのものもあれば、road1〜4まで４つあるものまで。
    # とりあえず、road1だけを採用し、接する道路数を表す項目を追加することにする。
    # Future TODO
    # road2〜road4のデータの採用のしかた
    
    # road1
    # road1_hkとroad1_sbをダミー変数化する。
    # road2〜road4の有無で、道路数を表すroad_numを作る
    # road2～road4の間口(road_mg)でトータルの間口を表すroad_mgを作る
    # TODO: road2〜road4を削除
    df = pd.merge(df, pd.get_dummies(df[['road1_hk','road1_sb']],drop_first = False), left_index = True, right_index = True)
    df['road_num']=df.apply(lambda x: 4 if x['road4_fi']!=0 else 3 if x['road3_fi']!=0 else 2 if x['road2_fi']!=0 else 1, axis=1)
    df['road_mg']=df['road2_mg']+df['road3_mg']+df['road4_mg']
    df['road_mg']=df['road_mg'].fillna(0)
    
    #kaoku_umとyheki_umuを0と1で置き換え
    df['kaoku_um']=df.apply(lambda x: 0 if x['kaoku_um']=="（無）" else 1, axis=1)
    df['yheki_umu']=df.apply(lambda x: 0 if x['yheki_umu']=="（無）" else 1, axis=1)
    
    #yheki_yohiを0と1で置き換え
    df['yheki_yohi']=df.apply(lambda x: 0 if x['yheki_yohi']=="（不要）" else 1, axis=1)
    
    #kborjsを0と1で置き換え。"公募"に変換ミスがあって、３通りあるため、「実測」で値を決める
    df['kborjs']=df.apply(lambda x: 0 if x['kborjs']=='実測' else 1, axis=1)
    
    # 引渡の状態 hw_status
    # 善し悪しを表しそうなので数値化したいが、とりあえずはダミー変数化する。
    df = pd.merge(df, pd.get_dummies(df[['hw_status']],drop_first = False), left_index = True, right_index = True)
    
    #都市計画区域別 toshikuiki1とtoshikuiki２　toshikuiki2があればそちらを採用する。
    #一度toshikuiki2がカラの場合にtoshikuiki(新設)に特殊な値を入れて比較しないとうまくいかない。他に方法がありそうだが
    df['toshikuiki']=df['toshikuiki2'].fillna('empty')
    df['toshikuiki'] = df.apply(lambda x: x['toshikuiki1'] if x['toshikuiki']=="empty" else x['toshikuiki2'], axis=1)
    df = pd.merge(df, pd.get_dummies(df[['toshikuiki']],drop_first = False), left_index = True, right_index = True)
    
    # 高度地区(kodochiku) ダミー変数化する
    df = pd.merge(df, pd.get_dummies(df[['kodochiku']],drop_first = False), left_index = True, right_index = True)
    
    # chikukeikaku, keikakuroad, kaihatsukyoka, t53kyoka, hokakyokaを0,1に変換
    df['chikukeikaku']=df.apply(lambda x: 0 if x['chikukeikaku']=="（無）" else 1, axis=1)
    df['keikakuroad']=df.apply(lambda x: 0 if x['keikakuroad']=="（無）" else 1, axis=1)
    df['kaihatsukyoka']=df.apply(lambda x: 0 if x['kaihatsukyoka']=="（不要）" else 1, axis=1)
    df['t53kyoka']=df.apply(lambda x: 0 if x['t53kyoka']=="（不要）" else 1, axis=1)
    df['hokakyoka']=df.apply(lambda x: 0 if x['hokakyoka']=="（不要）" else 1, axis=1)
    
    # 防火地区をダミー変数化
    df = pd.merge(df, pd.get_dummies(df[['bokachiiki']],drop_first = False), left_index = True, right_index = True)
    
    # 最低敷地面積(minmenseki) 欠損がある。ゼロで補完する
    df['minmenseki'] = df['minmenseki'].fillna(0)
    
    # その他規制①～④(hokakisei1～4)
    # 四つの列でダミー変数化する必要あり。
    
    # まず、表記を一意にしておき...
    kisei_map = {'公有地拡大推進法':'公拡法', '文化財保護法':'埋蔵文化財', '文化財保護法（埋蔵文化財）':'埋蔵文化財', '景観地区':'景観法','東日本大震災復興特別区域法':'東日本震災復興特', '農地法届出要':'農地法', '43条第1項ただし書き許可':'43条許可'}
    df['hokakisei1']=df['hokakisei1'].replace(kisei_map)
    df['hokakisei2']=df['hokakisei2'].replace(kisei_map)
    df['hokakisei3']=df['hokakisei3'].replace(kisei_map)
    df['hokakisei4']=df['hokakisei4'].replace(kisei_map)
       
    # カラのダミー変数列をつくっておいて...
    kisei_dict = {}
    for x in hokakisei_cols:
        kisei_dict[x] = np.zeros(len(df), dtype=np.int)
    
    # ダミー変数列を作って...
    for i in range(len(df)):
        name = df.at[i,'hokakisei1']
        if ( type(name) != float):
            kisei = kisei_dict[name]
            kisei[i]=1
        name = df.at[i,'hokakisei2']
        if ( type(name) != float):
            kisei = kisei_dict[name]
            kisei[i]=1
        name = df.at[i,'hokakisei3']
        if ( type(name) != float):
            kisei = kisei_dict[name]
            kisei[i]=1
        name = df.at[i,'hokakisei4']
        if ( type(name) != float):
            kisei = kisei_dict[name]
            kisei[i]=1
    
    # 元のDataFrameとダミー変数化したものを連結して..
    df = pd.concat([df, pd.DataFrame(kisei_dict)], axis=1)
    
    # 不要になったhokakisei1～hokakisei4を削除する
    df = df.drop(['hokakisei1','hokakisei2','hokakisei3','hokakisei4'], axis=1)
    
    # kinshijikoを0,1に変換
    df['kinshijiko']=df.apply(lambda x: 0 if x['kinshijiko']=="（無）" else 1, axis=1)
    
    # 路線価格(rosenka_hb)
    # 公示価格(koji_hb)
    # 基準地価(kijun_hb)
    # 路線価格と基準地価には欠損(値が0)あり。公示価格は欠損なし。
    # 公示価格だけを残す。
    # df = df.drop(['rosenka_hb','kijun_hb'],axis=1)

    # 有効宅地面積(mseki_yt_hb)
    # 道路(mseki_rd_hb)
    # ゴミ置場・公園等(mseki_dp_hb)
    # 相関不明。そのまま残しておく。
    
    # 土地最小面積～建物平均面積
    # tc_mseki_min_hb～tt_mseki_avg_hb
    # 地価に関係ないと思われるので落とす。
    # df = df.drop(['tc_mseki_min_hb','tc_mseki_max_hb','tt_mseki_min_hb','tt_mseki_max_hb','tc_mseki_avg_hb','tt_mseki_avg_hb'], axis=1)
    
    # 幅員4m未満道路の通過(fi4m_yohi)と幅員3m以上4m未満道路の通過距離(fi4m_kyori)
    # fi4m_yoshiが否なら、fi4m_kyoriが０。fi3m...の項目も同様。
    # よって、fi4m_yohiとfi3m_yohiは削除する。
    # かつ、fi4m_kyori, fi3m_kyoriが欠損しているところは、yohiがちゃんと否になっているので、0で埋める。
    df = df.drop(['fi4m_yohi','fi3m_yohi'],axis=1)
    df['fi4m_kyori']=df['fi4m_kyori'].fillna(0)
    df['fi3m_kyori']=df['fi3m_kyori'].fillna(0)
    
    # バス要否、バス本数
    # 後続の交通手段のところで、バスを使うことになっていないにもかかわらず、要否が要になっているデータがあり
    # 信ぴょう性に欠けるので、削除する。
    df = df.drop(['bus_yohi','bus_hon'], axis=1)
    
    # コンビニ有無(sho_conv)～騒音有無(shu_soon)
    # ２値データ。ありは「〇」。これを0/1に変換する。
    df['sho_conv'] = df['sho_conv'].map({'○':1, np.NaN:0})
    df['sho_super'] = df['sho_super'].map({'○':1, np.NaN:0})
    df['sho_shoten'] = df['sho_shoten'].map({'○':1, np.NaN:0})    
    df['sho_market'] = df['sho_market'].map({'○':1, np.NaN:0})
    df['shu_jutaku'] = df['shu_jutaku'].map({'○':1, np.NaN:0})
    df['shu_park'] = df['shu_park'].map({'○':1, np.NaN:0})
    df['shu_shop'] = df['shu_shop'].map({'○':1, np.NaN:0})
    df['shu_factory'] = df['shu_factory'].map({'○':1, np.NaN:0})
    df['shu_hvline'] = df['shu_hvline'].map({'○':1, np.NaN:0})
    df['shu_tower'] = df['shu_tower'].map({'○':1, np.NaN:0})
    df['shu_bochi'] = df['shu_bochi'].map({'○':1, np.NaN:0})
    df['shu_sogi'] = df['shu_sogi'].map({'○':1, np.NaN:0})
    df['shu_zoki'] = df['shu_zoki'].map({'○':1, np.NaN:0})
    df['shu_kokyo'] = df['shu_kokyo'].map({'○':1, np.NaN:0})
    df['shu_highway'] = df['shu_highway'].map({'○':1, np.NaN:0})
    df['shu_kaido'] = df['shu_kaido'].map({'○':1, np.NaN:0})
    df['shu_line_ari'] = df['shu_line_ari'].map({'○':1, np.NaN:0})
    df['shu_line_nashi'] = df['shu_line_nashi'].map({'○':1, np.NaN:0})
    df['shu_soon'] = df['shu_soon'].map({'○':1, np.NaN:0})
    
    # 幼稚園徒歩/小学校徒歩/中学校徒歩(gk_yoc_tm/gk_sho_tm/gk_chu_tm)
    # 欠損もなく異常値もないため、そのままにする。
    
    # 隣接物件東戸建（2F以下）～隣接物件北田畑
    # ２値データ。ありは「〇」。これを0/1に変換する
    df['rs_e_kdate2'] = df['rs_e_kdate2'].map({'○':1, np.NaN:0})
    df['rs_e_kdate3'] = df['rs_e_kdate3'].map({'○':1, np.NaN:0})
    df['rs_e_parking'] = df['rs_e_parking'].map({'○':1, np.NaN:0})
    df['rs_e_zoki'] = df['rs_e_zoki'].map({'○':1, np.NaN:0}) 
    df['rs_e_m_ari'] = df['rs_e_m_ari'].map({'○':1, np.NaN:0}) 
    df['rs_e_m_nashi'] = df['rs_e_m_nashi'].map({'○':1, np.NaN:0}) 
    df['rs_e_tahata'] = df['rs_e_tahata'].map({'○':1, np.NaN:0}) 

    df['rs_w_kdate2'] = df['rs_w_kdate2'].map({'○':1, np.NaN:0})
    df['rs_w_kdate3'] = df['rs_w_kdate3'].map({'○':1, np.NaN:0})
    df['rs_w_parking'] = df['rs_w_parking'].map({'○':1, np.NaN:0})
    df['rs_w_zoki'] = df['rs_w_zoki'].map({'○':1, np.NaN:0}) 
    df['rs_w_m_ari'] = df['rs_w_m_ari'].map({'○':1, np.NaN:0}) 
    df['rs_w_m_nashi'] = df['rs_w_m_nashi'].map({'○':1, np.NaN:0}) 
    df['rs_w_tahata'] = df['rs_w_tahata'].map({'○':1, np.NaN:0}) 
    
    df['rs_s_kdate2'] = df['rs_s_kdate2'].map({'○':1, np.NaN:0})
    df['rs_s_kdate3'] = df['rs_s_kdate3'].map({'○':1, np.NaN:0})
    df['rs_s_parking'] = df['rs_s_parking'].map({'○':1, np.NaN:0})
    df['rs_s_zoki'] = df['rs_s_zoki'].map({'○':1, np.NaN:0}) 
    df['rs_s_m_ari'] = df['rs_s_m_ari'].map({'○':1, np.NaN:0}) 
    df['rs_s_m_nashi'] = df['rs_s_m_nashi'].map({'○':1, np.NaN:0}) 
    df['rs_s_tahata'] = df['rs_s_tahata'].map({'○':1, np.NaN:0}) 
    
    df['rs_n_kdate2'] = df['rs_n_kdate2'].map({'○':1, np.NaN:0})
    df['rs_n_kdate3'] = df['rs_n_kdate3'].map({'○':1, np.NaN:0})
    df['rs_n_parking'] = df['rs_n_parking'].map({'○':1, np.NaN:0})
    df['rs_n_zoki'] = df['rs_n_zoki'].map({'○':1, np.NaN:0}) 
    df['rs_n_m_ari'] = df['rs_n_m_ari'].map({'○':1, np.NaN:0}) 
    df['rs_n_m_nashi'] = df['rs_n_m_nashi'].map({'○':1, np.NaN:0}) 
    df['rs_n_tahata'] = df['rs_n_tahata'].map({'○':1, np.NaN:0}) 

    # 路線(rosen_nm1/rosen_nm2)、駅(eki_nm1/eki_nm2)
    # 残してある公示価格に反映されていると考えて、ここでは削除する
    df[['bus', 'walk']] = df.apply(calculateLogistics, axis=1)
    df=df.drop(['rosen_nm1','eki_nm1','bas_toho1','eki_kyori1','bastei_nm1','teiho1','rosen_nm2','eki_nm2','bas_toho2','eki_kyori2','bastei_nm2','teiho2'],axis=1)

    # 最後に不要になった列を削除する
    # road_hkは、gotoデータと結合した後に補完に使うので、ここでは削除しないでおく。
    df=df.drop(['yoto1','kempei1','yoto2','kempei2','yoseki2','road1_sb','toshikuiki1','yoto'],axis=1)
    df=df.drop(['usui','road2_hk','road2_sb','road2_fi','road2_mg','road3_hk','road3_sb','road3_fi','road3_mg','road4_hk','road4_sb','road4_fi','road4_mg'],axis=1)
    df=df.drop(['hw_status','toshikuiki','toshikuiki2','toshikuiki','kodochiku','bokachiiki'],axis=1)
    df=df.drop(['gas', 'gk_sho_kyori', 'gk_chu_kyori'],axis=1)
    
    # testデータの項目gc_yoc_tmに欠損あり。
    # 平均値で欠損補完する
    df['gk_yoc_tm'] = df['gk_yoc_tm'].fillna(df['gk_yoc_tm'].mean())
        
    return df

def select_yoto(x):
    # yoto2はNaNを含む
    # 値がない場合、typeはfloatで、データがある場合にはstrになる様子。
    # そこで、型がstrかどうかで、値の有無を判定する。
    if ( type(x['yoto2']) == str ):
        yoto =x['yoto2']
        kempei = x['kempei2']
        yoseki =x['yoseki2']
    else:
        yoto =x['yoto1']
        kempei =x['kempei1']
        yoseki =x['yoseki1'] 
    return pd.Series([yoto, kempei, yoseki])

# 駅から徒歩、バス等の交通手段を、バスと徒歩の二つの利用時間に変換する
# バスを使わない場合には、busを0にすることで対応。

def calculateLogistics(x):
    if ( x['bas_toho1'] == 'バス' ):
        bus = x['eki_kyori1']
        walk = x['teiho1']
    else:
        bus = 0
        walk = x['eki_kyori1']
    
    return pd.Series([bus, walk])

# 法規制の処理用関数
def put_name(col, name):
    if ( type(name) != float ):
        col.add(name)

In [4]:
def preprocess_goto(goto, kobetsu_cols):
    # コピーしてから加工する
    df = goto.copy()
    
    # 削除対象のカラム名を追加していくリスト
    del_list = []
    
    # 階数・プラン(levelplan)
    # 求めるものは土地の販売価格なので不要とする。
    del_list.append("levelplan")
    
    # 幅員(fukuin)
    # 欠損および０の場合には、genbaデータと結合したroad1_fiの値をコピーする
    df['fukuin'] = df['fukuin'].fillna(0)
    for i in range(len(df)):
        if ( df.at[i,'fukuin'] ==0 ) :
            df.at[i,'fukuin'] = df.at[i,'road1_fi']
    
    # 道路状況(road_st)
    # 以下のように値にマップする。欠損は"問題なし”にマップすることにする。
    df['road_st'] = df['road_st'].fillna("問題なし")
    df['road_st'] = df['road_st'].map({'未舗装':0, '問題なし':1, '歩道あり':1, '歩道+緑地帯あり':2})
    
    # 間口(magutchi)
    # 幅員と同じように補完する。
    df['magutchi'] = df['magutchi'].fillna(0)
    for i in range(len(df)):
        if ( df.at[i,'magutchi'] ==0 ) :
            df.at[i,'magutchi'] = df.at[i,'road1_mg']    
    
    # 接道方位	(setsudo_hi)
    # 幅員と同じように補完してから、ダミー変数化する。
    df['setsudo_hi' ] = df['setsudo_hi'].fillna('unknown')
    for i in range(len(df)):
        if ( df.at[i,'setsudo_hi']=='unknown'):
            df.at[i,'setsudo_hi']=df.at[i,'road1_hk']

    # ダミー変数化。
    # 単にダミー変数化したあと、複数方向が一つに組み合わさっている場合があるので、分解する
    df['setsudo_hi']=df['setsudo_hi'].replace({'北+東':'北＋東', '北+西':'北＋西', '北西+南西':'北西＋南西','南東+南西':'南東＋南西'})
    df = pd.merge(df, pd.get_dummies(df[['setsudo_hi']],drop_first = False), left_index = True, right_index = True)
    # 全行処理する
    for i in range(len(df)):
        process_setsudo_hi(df,i)

    # 複数方向が組み合わさった列を削除する
    del_list.extend(['setsudo_hi_北東＋北西','setsudo_hi_北東＋南東','setsudo_hi_北東＋南西','setsudo_hi_北西＋北東'])
    del_list.extend(['setsudo_hi_北西＋南東','setsudo_hi_北西＋南西','setsudo_hi_北西＋東'])
    del_list.extend(['setsudo_hi_北＋北東','setsudo_hi_北＋北西','setsudo_hi_北＋南','setsudo_hi_北＋南東','setsudo_hi_北＋南西','setsudo_hi_北＋東','setsudo_hi_北＋西'])
    del_list.extend(['setsudo_hi_南東＋北','setsudo_hi_南東＋北東','setsudo_hi_南東＋北西','setsudo_hi_南東＋南西','setsudo_hi_南東＋北西'])
    del_list.extend(['setsudo_hi_南東＋南西','setsudo_hi_南東＋東','setsudo_hi_南東＋西'])
    del_list.extend(['setsudo_hi_南西＋北','setsudo_hi_南西＋北東','setsudo_hi_南西＋北西','setsudo_hi_南西＋南東','setsudo_hi_南西＋北','setsudo_hi_南西＋東','setsudo_hi_南西＋西'])
    del_list.extend(['setsudo_hi_南＋北','setsudo_hi_南＋北東','setsudo_hi_南＋北西','setsudo_hi_南＋南東','setsudo_hi_南＋南西','setsudo_hi_南＋東','setsudo_hi_南＋西'])
    del_list.extend(['setsudo_hi_東＋北','setsudo_hi_東＋南','setsudo_hi_東＋南東','setsudo_hi_東＋南西','setsudo_hi_東＋西'])
    del_list.extend(['setsudo_hi_西＋北','setsudo_hi_西＋北東','setsudo_hi_西＋北西','setsudo_hi_西＋東','setsudo_hi_西＋南東','setsudo_hi_西＋南西','setsudo_hi_西＋東'])
    del_list.extend(['setsudo_hi_北東＋南','setsudo_hi_北東＋西','setsudo_hi_北西＋南西＋北東','setsudo_hi_南西＋南','setsudo_hi_西＋南'])
    del_list.append('setsudo_hi')
    del_list.append('road1_hk')
    
    # 接道形状(setsudo_kj)
    # trainデータでは、'良い'が一番多いので、それで補完し、ダミー変数化する。
    df['setsudo_kj' ] = df['setsudo_kj'].fillna('良い')
    df = pd.merge(df, pd.get_dummies(df[['setsudo_kj']],drop_first = False), left_index = True, right_index = True)
    del_list.append('setsudo_kj')
    
    # 地型(jigata)
    # trainデータで一番多い'整形地’で補完し、ダミー変数化する。
    df['jigata'] = df['jigata'].fillna('整形地')
    df = pd.merge(df, pd.get_dummies(df[['jigata']],drop_first = False), left_index = True, right_index = True)
    del_list.append('jigata')
    
    # 日当たり(hiatari)
    # trainデータで多い'普通'で補完し、ダミー変数化する
    # genbaデータから補完できるかもしれない。
    df['hiatari'] = df['hiatari'].fillna('普通')
    df = pd.merge(df, pd.get_dummies(df[['hiatari']],drop_first = False), left_index = True, right_index = True)
    del_list.append('hiatari')
    
    # 個別要因①〜④
    # ダミー変数化する  
    kobetsu_map = {'信号近い':'信号前'}
    df['kobetsu1']=df['kobetsu1'].replace(kobetsu_map)
    df['kobetsu2']=df['kobetsu2'].replace(kobetsu_map)
    df['kobetsu3']=df['kobetsu3'].replace(kobetsu_map)
    df['kobetsu4']=df['kobetsu4'].replace(kobetsu_map)

    # ステップ２：
    kobetsu_dict = {}
    for x in kobetsu_cols:
        kobetsu_dict[x] = np.zeros(len(df), dtype=np.int)

    # ステップ３：
    for i in range(len(df)):
        name = df.at[i,'kobetsu1']
        if ( type(name) != float):
            kobetsu= kobetsu_dict[name]
            kobetsu[i]=1
        name = df.at[i,'kobetsu2']
        if ( type(name) != float):
            kobetsu= kobetsu_dict[name]
            kobetsu[i]=1
        name = df.at[i,'kobetsu3']
        if ( type(name) != float):
            kobetsu= kobetsu_dict[name]
            kobetsu[i]=1
        name = df.at[i,'kobetsu4']
        if ( type(name) != float):
            kobetsu= kobetsu_dict[name]
            kobetsu[i]=1

    # 元のDataFrameとダミー変数化したものを連結
    df = pd.concat([df, pd.DataFrame(kobetsu_dict)], axis=1)
    
    # 不要になった列を削除する
    del_list.append('kobetsu1')
    del_list.append('kobetsu2')
    del_list.append('kobetsu3')
    del_list.append('kobetsu4')
    
    df = df.drop(del_list, axis=1)
    
    # V5で追加するtt_msekiの補完処理。ここから。
    # 1. tt_mseki_avg_hb, tt_mseki_max_hb で補完
    df['tt_mseki_avg_hb'] = df['tt_mseki_avg_hb'].fillna(0)
    df['tt_mseki_max_hb'] = df['tt_mseki_max_hb'] .fillna(0)
    df['tt_mseki'] = df.apply(lambda x : x['tt_mseki'] if x['tt_mseki'] != 0 else x['tt_mseki_avg_hb'] if x['tt_mseki_avg_hb']!= 0 else x['tt_mseki_max_hb']  if x['tt_mseki_max_hb'] != 0 else 0, axis=1)

    # 1. tt_msekiがゼロ以外の集合に対して、容積率ごとに建物面積との比率を求める
    # 1-1 tt_mseki != 0の集合を作る
    # 1-2 容積率yoseki1でgroupbyする
    # 1-3 一つの容積率についてのDataFrameを取得
    # 1-4 当該DataFrameでtt_msekiとtc_msekiのそれぞれの合計値を求め、tt_mseki/tc_msekiの比ratioをともめる
    # 1-5 容積率とratioの組をdictに登録する
    df2 = df[df['tt_mseki']!=0]
    yoseki_gr = df2.groupby('yoseki1')
    ratio_dict = {}
    for y in yoseki_gr.groups.keys():
        # 1-3
        d = yoseki_gr.get_group(y)
        #1-4
        total_tc = d['tc_mseki'].sum()
        total_tt = d['tt_mseki'].sum()
        ratio = total_tt/total_tc
        #1-5
        ratio_dict[y]= ratio
    
    df['tt_mseki'] = df.apply(calc_tt_mseki, rdict = ratio_dict, axis=1)
    
    # V4では削除していた列。V5では一旦削除せず、`feature_importances_`を確認することにする
    # 土地最小面積～建物平均面積
    # tc_mseki_min_hb～tt_mseki_avg_hb
    # 地価に関係ないと思われるので落とす。
    # df = df.drop(['tc_mseki_min_hb','tc_mseki_max_hb','tt_mseki_min_hb','tt_mseki_max_hb','tc_mseki_avg_hb','tt_mseki_avg_hb'], axis=1)
    
    return df

def calc_tt_mseki( x , rdict ):
    if x['tt_mseki'] != 0:
        return x['tt_mseki']
    yoseki = x['yoseki1']
    ratio = rdict[yoseki]
    tt_mseki = x['tc_mseki'] * ratio
    return tt_mseki

def process_setsudo_hi(df,i):
    if df.at[i,'setsudo_hi_北東＋北西']==1:
        df.at[i,'setsudo_hi_北東']=1
        df.at[i,'setsudo_hi_北西']=1
    elif df.at[i,'setsudo_hi_北東＋南東']==1:
        df.at[i,'setsudo_hi_北東']=1
        df.at[i,'setsudo_hi_南東']=1
    elif df.at[i,'setsudo_hi_北東＋南西']==1:
        df.at[i,'setsudo_hi_北東']=1
        df.at[i,'setsudo_hi_南西']=1
    elif df.at[i,'setsudo_hi_北東＋南']==1:
        df.at[i,'setsudo_hi_北東']=1
        df.at[i,'setsudo_hi_南']=1
    elif df.at[i,'setsudo_hi_北東＋西']==1:
        df.at[i,'setsudo_hi_北東']=1
        df.at[i,'setsudo_hi_西']=1
    elif df.at[i,'setsudo_hi_北西＋北東']==1:
        df.at[i,'setsudo_hi_北西']=1
        df.at[i,'setsudo_hi_北東']=1
    elif df.at[i,'setsudo_hi_北西＋南東']==1:
        df.at[i,'setsudo_hi_北西']=1
        df.at[i,'setsudo_hi_南東']=1
    elif df.at[i,'setsudo_hi_北西＋南西']==1:
        df.at[i,'setsudo_hi_北西']=1
        df.at[i,'setsudo_hi_南西']=1
    elif df.at[i,'setsudo_hi_北西＋南西＋北東']==1:
        df.at[i,'setsudo_hi_北西']=1
        df.at[i,'setsudo_hi_南西']=1
        df.at[i,'setsudo_hi_北東']=1
    elif df.at[i,'setsudo_hi_北西＋東']==1:
        df.at[i,'setsudo_hi_北西']=1
        df.at[i,'setsudo_hi_東']=1
    elif df.at[i,'setsudo_hi_北＋北東']==1:
        df.at[i,'setsudo_hi_北']=1
        df.at[i,'setsudo_hi_北東']=1
    elif df.at[i,'setsudo_hi_北＋北西']==1:
        df.at[i,'setsudo_hi_北']=1
        df.at[i,'setsudo_hi_北西']=1
    elif df.at[i,'setsudo_hi_北＋南']==1:
        df.at[i,'setsudo_hi_北']=1
        df.at[i,'setsudo_hi_南']=1
    elif df.at[i,'setsudo_hi_北＋南東']==1:
        df.at[i,'setsudo_hi_北']=1
        df.at[i,'setsudo_hi_南東']=1
    elif df.at[i,'setsudo_hi_北＋南西']==1:
        df.at[i,'setsudo_hi_北']=1
        df.at[i,'setsudo_hi_南西']=1
    elif df.at[i,'setsudo_hi_北＋東']==1:
        df.at[i,'setsudo_hi_北']=1
        df.at[i,'setsudo_hi_東']=1
    elif df.at[i,'setsudo_hi_北＋西']==1:
        df.at[i,'setsudo_hi_北']=1
        df.at[i,'setsudo_hi_西']=1
    elif df.at[i,'setsudo_hi_南東＋北']==1:
        df.at[i,'setsudo_hi_南東']=1
        df.at[i,'setsudo_hi_北']=1
    elif df.at[i,'setsudo_hi_南東＋北東']==1:
        df.at[i,'setsudo_hi_南東']=1
        df.at[i,'setsudo_hi_北東']=1
    elif df.at[i,'setsudo_hi_南東＋北西']==1:
        df.at[i,'setsudo_hi_南東']=1
        df.at[i,'setsudo_hi_北西']=1
    elif df.at[i,'setsudo_hi_南東＋南西']==1:
        df.at[i,'setsudo_hi_南東']=1
        df.at[i,'setsudo_hi_南西']=1
    elif df.at[i,'setsudo_hi_南東＋北西']==1:
        df.at[i,'setsudo_hi_南東']=1
        df.at[i,'setsudo_hi_北西']=1
    elif df.at[i,'setsudo_hi_南東＋南西']==1:
        df.at[i,'setsudo_hi_南東']=1
        df.at[i,'setsudo_hi_南西']=1
    elif df.at[i,'setsudo_hi_南東＋東']==1:
        df.at[i,'setsudo_hi_南東']=1
        df.at[i,'setsudo_hi_東']=1
    elif df.at[i,'setsudo_hi_南東＋西']==1:
        df.at[i,'setsudo_hi_南東']=1
        df.at[i,'setsudo_hi_西']=1
    elif df.at[i,'setsudo_hi_南西＋北']==1:
        df.at[i,'setsudo_hi_南西']=1
        df.at[i,'setsudo_hi_北']=1
    elif df.at[i,'setsudo_hi_南西＋北東']==1:
        df.at[i,'setsudo_hi_南西']=1
        df.at[i,'setsudo_hi_北東']=1
    elif df.at[i,'setsudo_hi_南西＋北西']==1:
        df.at[i,'setsudo_hi_南西']=1
        df.at[i,'setsudo_hi_北西']=1
    elif df.at[i,'setsudo_hi_南西＋南東']==1:
        df.at[i,'setsudo_hi_南西']=1
        df.at[i,'setsudo_hi_南東']=1
    elif df.at[i,'setsudo_hi_南西＋北']==1:
        df.at[i,'setsudo_hi_南西']=1
        df.at[i,'setsudo_hi_北']=1
    elif df.at[i,'setsudo_hi_南西＋東']==1:
        df.at[i,'setsudo_hi_南西']=1
        df.at[i,'setsudo_hi_東']=1
    elif df.at[i,'setsudo_hi_南西＋西']==1:
        df.at[i,'setsudo_hi_南西']=1
        df.at[i,'setsudo_hi_西']=1
    elif df.at[i,'setsudo_hi_南西＋南']==1:
        df.at[i,'setsudo_hi_南西']=1
        df.at[i,'setsudo_hi_南']=1
    elif df.at[i,'setsudo_hi_南＋北']==1:
        df.at[i,'setsudo_hi_南']=1
        df.at[i,'setsudo_hi_北']=1
    elif df.at[i,'setsudo_hi_南＋北東']==1:
        df.at[i,'setsudo_hi_南']=1
        df.at[i,'setsudo_hi_北東']=1
    elif df.at[i,'setsudo_hi_南＋北西']==1:
        df.at[i,'setsudo_hi_南']=1
        df.at[i,'setsudo_hi_北西']=1
    elif df.at[i,'setsudo_hi_南＋南東']==1:
        df.at[i,'setsudo_hi_南']=1
        df.at[i,'setsudo_hi_南東']=1
    elif df.at[i,'setsudo_hi_南＋南西']==1:
        df.at[i,'setsudo_hi_南']=1
        df.at[i,'setsudo_hi_南西']=1
    elif df.at[i,'setsudo_hi_南＋東']==1:
        df.at[i,'setsudo_hi_南']=1
        df.at[i,'setsudo_hi_東']=1
    elif df.at[i,'setsudo_hi_南＋西']==1:
        df.at[i,'setsudo_hi_南']=1
        df.at[i,'setsudo_hi_西']=1
    elif df.at[i,'setsudo_hi_東＋北']==1:
        df.at[i,'setsudo_hi_東']=1
        df.at[i,'setsudo_hi_北']=1
    elif df.at[i,'setsudo_hi_東＋南']==1:
        df.at[i,'setsudo_hi_東']=1
        df.at[i,'setsudo_hi_南']=1
    elif df.at[i,'setsudo_hi_東＋南東']==1:
        df.at[i,'setsudo_hi_東']=1
        df.at[i,'setsudo_hi_南東']=1
    elif df.at[i,'setsudo_hi_東＋南西']==1:
        df.at[i,'setsudo_hi_東']=1
        df.at[i,'setsudo_hi_南西']=1
    elif df.at[i,'setsudo_hi_東＋西']==1:
        df.at[i,'setsudo_hi_東']=1
        df.at[i,'setsudo_hi_西']=1
    elif df.at[i,'setsudo_hi_西＋北']==1:
        df.at[i,'setsudo_hi_西']=1
        df.at[i,'setsudo_hi_北']=1
    elif df.at[i,'setsudo_hi_西＋北東']==1:
        df.at[i,'setsudo_hi_西']=1
        df.at[i,'setsudo_hi_北東']=1
    elif df.at[i,'setsudo_hi_西＋北西']==1:
        df.at[i,'setsudo_hi_西']=1
        df.at[i,'setsudo_hi_北西']=1
    elif df.at[i,'setsudo_hi_西＋東']==1:
        df.at[i,'setsudo_hi_西']=1
        df.at[i,'setsudo_hi_東']=1
    elif df.at[i,'setsudo_hi_西＋南東']==1:
        df.at[i,'setsudo_hi_西']=1
        df.at[i,'setsudo_hi_南東']=1
    elif df.at[i,'setsudo_hi_西＋南西']==1:
        df.at[i,'setsudo_hi_西']=1
        df.at[i,'setsudo_hi_南西']=1
    elif df.at[i,'setsudo_hi_西＋東']==1:
        df.at[i,'setsudo_hi_西']=1
        df.at[i,'setsudo_hi_東']=1
    elif df.at[i,'setsudo_hi_西＋南']==1:
        df.at[i,'setsudo_hi_西']=1
        df.at[i,'setsudo_hi_南']=1
    return


In [5]:
# 1. 現場データのtrainとtestを連結
train = pd.read_csv("data/train_genba.tsv", sep='\t')
test = pd.read_csv("data/test_genba.tsv", sep='\t')

# 2. 連結した現場データの欠損補完やhokakisei1～4のダミー変数化を行う
genba = pd.concat([train,test],axis=0, ignore_index=True)
hokakisei_list = create_list( genba, {'hokakisei1','hokakisei2','hokakisei3','hokakisei4'})
processed_genba = preprocess_genba(genba, hokakisei_list)

# 3. 号棟データのtrainを読みこみ、目的変数を分離しておく。
train = pd.read_csv("data/train_goto.tsv",sep='\t')
train_target = pd.DataFrame(train[['id','keiyaku_pr']])
train = train.drop(['keiyaku_pr'],axis=1)

# 4. 号棟データのtestを読み込み、上記のtrain号棟データと連結
test = pd.read_csv("data/test_goto.tsv",sep='\t')
goto = pd.concat([train,test])

# 5. 連結済みの現場データと連結済み号棟データをジョイン
joined_goto = goto.merge(processed_genba, how='left', on='pj_no' )

# 6. ジョイン済号棟データの欠損補完やkobetsu1～4のダミー変数化を行う
kobetsu_list = create_list( joined_goto,  ['kobetsu1','kobetsu2','kobetsu3','kobetsu4'])
processed_goto = preprocess_goto(joined_goto, kobetsu_list)

# 7. 号棟データのtrainとtestを分離
train_goto = processed_goto[processed_goto['id'].str.contains('train_')]
test_goto = processed_goto[processed_goto['id'].str.contains('test_')]

# 8. 号棟データのtrainから分離した真の目的変数から、学習で使用する単価面積を算出
# train_target['tanka_pr']=train_target['keiyaku_pr']/train_target['tc_mseki']

# 9. 処理結果をファイル出力
# processed_train_goto_x.csv ... 教師データ　説明変数
# processed_train_goto_y.csv ... 教師データ　目的変数
# processed_test_goto_x.csv ... 予測用データ
train_goto.to_csv('data/processed_train_goto_x_v7.csv', index=False)
test_goto.to_csv('data/processed_test_goto_x_v7.csv', index=False)
train_target.to_csv('data/processed_train_goto_y_v7.csv', index=False)

In [6]:
## 以上